# 1. Import relevant packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from keras.layers import Dense, Activation, LSTM
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
df=pd.read_csv('gdrive/My Drive/Colab Notebooks/hm_train.csv') # Reading the csv file from google drive folder
df.head()

,hmid,reflection_period,cleaned_hm,num_sentence,predicted_category
0,27673,24h,I went on a successful date with someone I fel...,1,affection
1,27674,24h,I was happy when my son got 90% marks in his e...,1,affection
2,27675,24h,I went to the gym this morning and did yoga.,1,exercise
3,27676,24h,We had a serious talk with some friends of our...,2,bonding
4,27677,24h,I went with grandchildren to butterfly display...,1,affection


In [4]:
import nltk
nltk.download("popular")
stopwords = nltk.corpus.stopwords.words('english') # Has all the stopwords in english

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

### 2. Cleaning the text

In [0]:
def clean_text(text):
    # It first checks for the punctuation characters and removes it. Next it converts the char to lowercase and joins the chars to form words.
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    text = ' '.join(word for word in text.split() if word not in stopwords)
    # This \W+ regex, indicates that it will split wherever it sees one or more non-word characters.
    tokens = re.split('\W+',text)
    # removing stopwords and stemming at a time
    #text = [word for word in tokens if word not in stopwords]
    return text

In [6]:
#Applying the clean_text function to the cleaned_hm column
df['cleaned_text'] = df.cleaned_hm.apply(lambda x: clean_text(x))
df.head()

,hmid,reflection_period,cleaned_hm,num_sentence,predicted_category,cleaned_text
0,27673,24h,I went on a successful date with someone I fel...,1,affection,went successful date someone felt sympathy con...
1,27674,24h,I was happy when my son got 90% marks in his e...,1,affection,happy son got 90 marks examination
2,27675,24h,I went to the gym this morning and did yoga.,1,exercise,went gym morning yoga
3,27676,24h,We had a serious talk with some friends of our...,2,bonding,serious talk friends flaky lately understood g...
4,27677,24h,I went with grandchildren to butterfly display...,1,affection,went grandchildren butterfly display crohn con...


### 3. Tokenize and find out unique words

In [7]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each row(cleaned_hm).
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['cleaned_text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 23001 unique tokens.


### 4. Convert text to sequences and padding 

In [15]:
X = tokenizer.texts_to_sequences(df['cleaned_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (60321, 250)


In [16]:
X[8888]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [17]:
y = pd.get_dummies(df['predicted_category']).values # One hot encoding for the Y variable
print('Shape of label tensor:', y.shape)

Shape of label tensor: (60321, 7)


### 5. Split the data into train & test sets 

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(54288, 250) (54288, 7)
(6033, 250) (6033, 7)


### 6. Build a sequencial model and complie it

In [20]:
model= Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 707       
Total params: 5,081,107
Trainable params: 5,081,107
Non-trainable params: 0
_________________________________________________________________
None


### 7.Fit the model on train data

In [21]:
epochs = 5
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 48859 samples, validate on 5429 samples
Epoch 1/5
48859/48859 [==============================] - 425s 9ms/step - loss: 0.6906 - acc: 0.7617 - val_loss: 0.3566 - val_acc: 0.8786
Epoch 2/5
48859/48859 [==============================] - 423s 9ms/step - loss: 0.2659 - acc: 0.9088 - val_loss: 0.3010 - val_acc: 0.8904
Epoch 3/5
48859/48859 [==============================] - 422s 9ms/step - loss: 0.1700 - acc: 0.9411 - val_loss: 0.3074 - val_acc: 0.8959
Epoch 4/5
48859/48859 [==============================] - 425s 9ms/step - loss: 0.1251 - acc: 0.9563 - val_loss: 0.3268 - val_acc: 0.8945
Epoch 5/5
48859/48859 [==============================] - 426s 9ms/step - loss: 0.0998 - acc: 0.9656 - val_loss: 0.3445 - val_acc: 0.8928


### 8.Evaluate on test data

In [22]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

6033/6033 [==============================] - 36s 6ms/step
Test set
  Loss: 0.389
  Accuracy: 0.887


### Predict the Category of Happiness on Unseen data(hm_test)

In [0]:
df_test =pd.read_csv('gdrive/My Drive/Colab Notebooks/hm_test.csv')

In [31]:
df_test.head()

,hmid,reflection_period,cleaned_hm,num_sentence
0,88305,3m,I spent the weekend in Chicago with my friends.,1
1,88306,3m,We moved back into our house after a remodel. ...,2
2,88307,3m,My fiance proposed to me in front of my family...,1
3,88308,3m,I ate lobster at a fancy restaurant with some ...,1
4,88309,3m,I went out to a nice restaurant on a date with...,5


In [32]:
#Applying the clean_text function to the cleaned_hm column
df_test['cleaned_text'] = df_test.cleaned_hm.apply(lambda x: clean_text(x))
df_test.head()

,hmid,reflection_period,cleaned_hm,num_sentence,cleaned_text
0,88305,3m,I spent the weekend in Chicago with my friends.,1,spent weekend chicago friends
1,88306,3m,We moved back into our house after a remodel. ...,2,moved back house remodel lived hotel 9 months ...
2,88307,3m,My fiance proposed to me in front of my family...,1,fiance proposed front family beginning march
3,88308,3m,I ate lobster at a fancy restaurant with some ...,1,ate lobster fancy restaurant friends
4,88309,3m,I went out to a nice restaurant on a date with...,5,went nice restaurant date wife popular restaur...


In [33]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each row(cleaned_hm).
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_test['cleaned_text'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 18460 unique tokens.


In [34]:
X_new = tokenizer.texts_to_sequences(df_test['cleaned_text'].values)
X_new = pad_sequences(X_new, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_new.shape)

Shape of data tensor: (40213, 250)


In [0]:
labels = ['achievement', 'affection', 'bonding', 'enjoy_the_moment', 'exercise',
       'leisure', 'nature']

In [0]:
X_DF = pd.DataFrame(X_new)

In [46]:
X_DF.iloc[:2] # checking the input shape

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,113,96,2020,16


In [0]:
pred_new_data = model.predict(X_DF) # predicting for the whole data 

In [57]:
pd.DataFrame(pred_new_data) #probabilities for all the classes, 7 columns are nothing but classes.

,0,1,2,3,4,5,6
0,0.000864,0.998818,0.000018,0.000277,3.170371e-06,0.000018,2.411445e-06
1,0.991206,0.000349,0.002237,0.005730,1.134637e-04,0.000204,1.602407e-04
2,0.990685,0.000822,0.001600,0.006183,1.142284e-04,0.000211,3.858813e-04
3,0.000036,0.999946,0.000003,0.000004,1.261579e-07,0.000011,2.066169e-07
4,0.165953,0.001465,0.005591,0.001370,4.592930e-04,0.824954,2.064804e-04
...,...,...,...,...,...,...,...
40208,0.032383,0.095041,0.012125,0.730514,1.369833e-02,0.109740,6.499065e-03
40209,0.742372,0.120104,0.019097,0.023164,6.912437e-02,0.007811,1.832702e-02
40210,0.011982,0.983128,0.002688,0.001042,3.478006e-04,0.000284,5.283003e-04
40211,0.999856,0.000021,0.000052,0.000062,4.707286e-06,0.000002,2.054863e-06


In [58]:
len(pred_new_data)

40213

In [0]:
# Creating a list to store the classes for each sentence
predicted_category = []
# Looping through the array sentences one by one to find out the maximum probability class and append it to the predicted_category variable 
for i in range(len(pred_new_data)):
    p = labels[np.argmax(pred_new_data[i])]
    predicted_category.append(p)

In [0]:
sub = pd.DataFrame({'hmid':df_test.hmid,'predicted_category':predicted_category})

In [61]:
sub.head()

,hmid,predicted_category
0,88305,affection
1,88306,achievement
2,88307,achievement
3,88308,affection
4,88309,leisure


In [62]:
sub.predicted_category.value_counts()

achievement         15300
affection           12731
enjoy_the_moment     4739
bonding              4645
leisure              2341
nature                264
exercise              193
Name: predicted_category, dtype: int64

# Observation :

### By using LSTM as the neurons for the hidden layers, I built the deep neural network model and got 88.7% accuracy on the test data.The LinearSVC model is performing better than the LSTMs model on the test data. So, the best model to predict the happiness source is LinearSVC.